<a href="https://colab.research.google.com/github/ChintPatel/CMPE255-Decision-trees-and-ensemble/blob/main/random_forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Model-Definition" data-toc-modified-id="Model-Definition-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Model Definition</a></span></li><li><span><a href="#Training-and-testing-on-Diabetes-Dataset" data-toc-modified-id="Training-and-testing-on-Diabetes-Dataset-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Training and testing on Diabetes Dataset</a></span></li><li><span><a href="#Metrics" data-toc-modified-id="Metrics-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Metrics</a></span></li></ul></div>

In [15]:
import numpy as np

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score

import pandas as pd

from collections import Counter
from sklearn.tree import DecisionTreeClassifier


# Model Definition

Using previously written Decision tree

In [16]:
def bootstrap_sample(X, y):
    n_samples = X.shape[0]
    idxs = np.random.choice(n_samples, n_samples, replace=True)
    return X[idxs], y[idxs]


def most_common_label(y):
    counter = Counter(y)
    most_common = counter.most_common(1)[0][0]
    return most_common

In [24]:
class RandomForest:
    def __init__(self, n_trees, max_depth, n_feats, min_samples_split=2):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.n_feats = n_feats
        self.min_samples_split = min_samples_split
        self.trees = []

    def bootstrap_sample(self, X, y):
        # Generate a bootstrap sample
        n_samples = X.shape[0]
        idxs = np.random.choice(n_samples, n_samples, replace=True)
        return X[idxs], y[idxs]

    def fit(self, X, y):
        self.trees = []
        for _ in range(self.n_trees):
            # Create a decision tree
            tree = DecisionTreeClassifier(max_depth=self.max_depth)
            X_samp, y_samp = self.bootstrap_sample(X, y)
            tree.fit(X_samp, y_samp)
            self.trees.append(tree)

    def predict(self, X):
        # Aggregate predictions from all trees
        tree_preds = np.array([tree.predict(X) for tree in self.trees])
        return np.apply_along_axis(lambda x: Counter(x).most_common(1)[0][0], axis=0, arr=tree_preds)

In [25]:
def accuracy(y_true, y_pred):
    accuracy = np.sum(y_true == y_pred) / len(y_true)
    return accuracy

# Training and testing on Diabetes Dataset

In [26]:
cols = ["Pregnancies", "Glucose", "BloodPressure", "SkinThickness",
        "Insulin", "BMI", "DiabetesPedigreeFunction", "Age", "Outcome"]
url = "https://gist.githubusercontent.com/ktisha/c21e73a1bd1700294ef790c56c8aec1f/raw/819b69b5736821ccee93d05b51de0510bea00294/pima-indians-diabetes.csv"
diabetes_data = pd.read_csv(url, skiprows=9, header=None, names=cols)

In [27]:
diabetes_data.shape

(768, 9)

In [28]:
X = diabetes_data[cols[:-1]].values
y = diabetes_data[cols[-1]].values

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y, random_state=42)

In [30]:
rnd_clf = RandomForest(max_depth=10, n_trees=10, n_feats=6)
rnd_clf.fit(X_train, y_train)
y_pred = rnd_clf.predict(X_test)

# Metrics

In [31]:
print(f"Accuracy: {accuracy(y_test, y_pred)}")

Accuracy: 0.7012987012987013


In [32]:
print(f"Precision: {precision_score(y_test, y_pred)}")
print(f"Recall: {recall_score(y_test, y_pred)}")
print(f"F1-Score: {f1_score(y_test, y_pred)}")

Precision: 0.5833333333333334
Recall: 0.5185185185185185
F1-Score: 0.5490196078431373
